In [1]:
!pip install beautifulsoup4 requests pandas

In [23]:
import urllib
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import pickle

In [26]:
def get_cna_articles():
    CNA_URL = "https://www.channelnewsasia.com/"
    # get cna main page

    page = requests.get(CNA_URL)
    soup = BeautifulSoup(page.text, "html.parser")
    # get links to different news articles (the class is for this url, will differ across websites)
    links = soup.find_all('a', attrs={'class': 'h6__link h6__link-- list-object__heading-link'})
    articles = []
    for link in links:
        article_page = requests.get(CNA_URL + link['href'])
        article = BeautifulSoup(article_page.text, "html.parser")
        article_text = article.find_all('div', attrs={'class': 'text-long'})
        articles.append(' '.join([[' '.join([child.text for child in div.findChildren('p')])][0] for div in article_text]))
    
    return articles

In [41]:
# run this block to get saved data/scrape data
CNA_ARTICLES_ADDRESS = Path("cna_articles.pickle")
if CNA_ARTICLES_ADDRESS.is_file():
    with open(CNA_ARTICLES_ADDRESS, "rb") as f:
        cna_articles = pickle.load(f)

else:
    cna_articles = get_cna_articles()
    CNA_ARTICLES_ADDRESS.touch(exist_ok=True)
    with open(CNA_ARTICLES_ADDRESS, "wb") as f:
        pickle.dump(cna_articles, f)

In [42]:
for article in articles:
    print(f"{article}\n\n")

BENGALURU :     Indian crypto asset platform CoinSwitch Kuber said on Wednesday it has raised more than US$260 million in a funding round from investors including Coinbase Ventures and Andreessen Horowitz, valuing it at US$1.9 billion.    (Reporting by Rama Venkat in Bengaluru; Editing by Subhranshu Sahu)


BRUSSELS :     Belgium captain Eden Hazard has a stage at the Nations League finals this week to make up for the setbacks of recent years as he struggled for form following a series of niggling injuries. The 30-year-old attacker is a long-time mainstay of his national team and one of their "golden generation" https://www.reuters.com/article/uk-soccer-uefanations-bel-fra-witsel/soccer-nations-league-an-opportunity-for-belgiums-golden-generation-witsel-idUKKBN2GU1ZC, but in the last two seasons his career has been stagnant, despite a dream move to Real Madrid. His playing time was restricted by a litany of complaints, mostly after surgery on his ankle but also including various muscle

In [32]:
CNA_ARTICLES_ADDRESS.is_file()

False

In [83]:
# scrape stackoverflow
from IPython.core.display import display, HTML
def get_stack_overflow_articles():
    STACK_OVERFLOW_URL = "https://stackoverflow.com/"
    page = requests.get(f"{STACK_OVERFLOW_URL}questions?tab=Frequent")
    soup = BeautifulSoup(page.text, "html.parser")
    # get links to different news articles (the class is for this url, will differ across websites)
    links = soup.find_all("a", "question-hyperlink")
    posts = []
    for idx, link in enumerate(links,1):
        article_site = requests.get(f"{STACK_OVERFLOW_URL}{link['href']}")
        article_page = BeautifulSoup(article_site.text, "html.parser")
        parent = article_page.find("div", "s-prose js-post-body")
        p_tags = parent.findChildren("p")
        post = []
        for p_tag in p_tags:
            post.append(p_tag.text)
            display(HTML(str(p_tag)))
        posts.append(" ".join(post))
        if idx == 50: break
    return posts

In [84]:
# run this block to get saved data/scrape data
STACK_OVERFLOW_ARTICLES_ADDRESS = Path("stack_overflow_articles.pickle")
if STACK_OVERFLOW_ARTICLES_ADDRESS.is_file():
    with open(STACK_OVERFLOW_ARTICLES_ADDRESS, "rb") as f:
        stack_overflow_articles = pickle.load(f)

else:
    stack_overflow_articles = get_stack_overflow_articles()
    STACK_OVERFLOW_ARTICLES_ADDRESS.touch(exist_ok=True)
    with open(STACK_OVERFLOW_ARTICLES_ADDRESS, "wb") as f:
        pickle.dump(stack_overflow_articles, f)

In [86]:
stack_overflow_articles

['When discussing performance with colleagues, teaching, sending a bug report or searching for guidance on mailing lists and here on Stack\xa0Overflow, a reproducible example is often asked and always helpful. What are your tips for creating an excellent example? How do you paste data structures from r in a text format? What other information should you include? Are there other tricks in addition to using dput(), dump() or structure()?  When should you include library() or require() statements?  Which reserved words should one avoid, in addition to c, df, data, etc.? How does one make a great r reproducible example?',
 'What are Null Pointer Exceptions (java.lang.NullPointerException) and what causes them? What methods/tools can be used to determine the cause so that you stop the exception from causing the program to terminate prematurely?',
 "I have a function foo which makes an asynchronous request. How can I return the response/result from foo? I am trying to return the value from t

In [104]:
# scrape hardwarezone
def get_hwz_articles():
    HWZ_URL = "https://forums.hardwarezone.com.sg/"
    page = requests.get(HWZ_URL)
    soup = BeautifulSoup(page.text, "html.parser")
    # get links to different news articles (the class is for this url, will differ across websites)
    divs = soup.find_all("div", "structItem-title")
    content = []
    for div in divs:
        forum = BeautifulSoup(requests.get(f"{HWZ_URL}{div.find('a')['href']}").text, "html.parser")
#         parent = article_page.find("div", "s-prose js-post-body")
        posts = [comment.text for comment in forum.find_all("div", "bbWrapper")]
        content.append(posts)
    return content

In [106]:
# run this block to get saved data/scrape data
HWZ_ARTICLES_PATH = Path("hwz_articles.pickle")
if HWZ_ARTICLES_PATH.is_file():
    with open(HWZ_ARTICLES_PATH, "rb") as f:
        hwz_articles = pickle.load(f)

else:
    hwz_articles = get_hwz_articles()
    HWZ_ARTICLES_PATH.touch(exist_ok=True)
    with open(HWZ_ARTICLES_PATH, "wb") as f:
        pickle.dump(hwz_articles, f)